In [1]:
import itertools
import os
import zipfile
import csv
import textwrap
import time
import multiprocessing

import importlib
# To reload a module without having to 
# restart the kernel, use importlib.reload
# For example -- 

# phasor = importlib.reload(phasor)

from difflib import SequenceMatcher
from pprint import pprint
from collections import Counter, deque
from time import time

import spacy
import numpy
import pandas
import umap

from IPython.display import display, Markdown

from scipy.spatial import cKDTree
from sklearn.neighbors import BallTree
from pyhash import city_64

import phasor
from headless import load_pages

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, TapTool, OpenURL, ColumnDataSource
from bokeh.palettes import magma

output_notebook()

Loading BokehJS ...

In [3]:
dataset = 'test-batch'
dataset = 'rand-fiction-1k'
dataset = 'pd-ic-combined'

secure_paths = (f'/media/secure_volume/volumes/zip/{dataset}',
                f'/media/secure_volume/derived/{dataset}/fft',
                f'/media/secure_volume/derived/{dataset}/srp_fft',
                f'/media/secure_volume/worksets/{dataset}-hathifiles.csv')

test_paths = ('../ht-open-test-data/fiction_998/',
              '../ht-open-test-data/fiction_fft',
              '../ht-open-test-data/fiction_srp_fft',
              '../ht-open-test-data/fiction.csv')

spe_paths = (f'../spevecs/volumes/{dataset}',
             f'../spevecs/derived/{dataset}/fft',
             f'../spevecs/derived/{dataset}/srp_fft',
             f'../spevecs/worksets/{dataset}-hathifiles.csv')

(path_to_volumes, 
 path_to_fft, 
 path_to_srp_fft, 
 path_to_meta) = spe_paths

In [4]:
# To load signatures for all 200k volumes, it currently takes 
# about an hour. It should be possible to reduce that.

n_signatures = 20000
start = time()
signature_data, metadata = phasor.load_fft_signature_metadata(
    path_to_fft, path_to_meta, reload=False,
    end=20, csv_delim=',', n_docs=n_signatures
)
end = time()
end - start

/Users/enderlej/.virtualenvs/th/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,4,10,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Reloading FFT signature data. This could take a while.


197.69862413406372

In [10]:
psig = phasor.PhasorSignature()
    
blob_copies = []
for i in range(100):
    start, offset, noisy = perturb_phasor_blob(data, index=i, n=2)
    blob_copies.extend(noisy)
blob_copies = numpy.array(blob_copies)

signatures = psig.transform(blob_copies)

same_diffs = [a - b for a, b in zip(signatures[0::2], signatures[1::2])]
diff_diffs = [a - b for a, b in zip(signatures[1::2], signatures[2::2])]
same_mags = [mag for d in same_diffs for mag in (d * d.conjugate()) ** 0.5]
diff_mags = [mag for d in diff_diffs for mag in (d * d.conjugate()) ** 0.5]

print('Another test we can perform is to look at the signatures ')
print('from data that comes from the same starting point, and see' )
print('how close together they are, and then compoare those to ')
print('signatures from data that comes from different starting points. ')
print()
print('This directly examines the metric we will be using to find ')
print('duplicates, so this had better be doing something right! ')
print()
print('These measure the (effectively euclidean) distance between ')
print('signatures based on data from the same starting point, rotated ')
print('and with noise added. So we expect these values...')
print()
print('The largest distance:', max(same_mags))
print('The average distance:', numpy.mean(same_mags))
print()
print('...to be significantly smaller than these values, which')
print('measure the distance between signatures based on data')
print('from different starting points.')
print()
print('The largest distance:', max(diff_mags))
print('The average distance:', numpy.mean(diff_mags))
print()
print('If that expectation is satisfied, then we can feel confident')
print('that this signature is going to do a pretty good job identifying')
print('texts that have similar underlying content, even if that content')
print('has been modified in various ways, such as by adding front and')
print('end matter, or through the inclusion of OCR errors and textual')
print('variants.')

Another test we can perform is to look at the signatures 
from data that comes from the same starting point, and see
how close together they are, and then compoare those to 
signatures from data that comes from different starting points. 

This directly examines the metric we will be using to find 
duplicates, so this had better be doing something right! 

These measure the (effectively euclidean) distance between 
signatures based on data from the same starting point, rotated 
and with noise added. So we expect these values...

The largest distance: 1.5385528853133739
The average distance: 0.14366908094608868

...to be significantly smaller than these values, which
measure the distance between signatures based on data
from different starting points.

The largest distance: 3.9257534880156904
The average distance: 0.5303210488731611

If that expectation is satisfied, then we can feel confident
that this signature is going to do a pretty good job identifying
texts that have similar under

In [7]:
# dedupe_slices[0].n_points

print("size of signature_data in megabytes:", signature_data.nbytes / 1000000)
print("number of rows in signature_data:", len(signature_data))
print(signature_data.shape)

size of signature_data in megabytes: 16.0
number of rows in signature_data: 20000
(20000, 100)


In [8]:
kwargs = dict(
    n_neighbors=10, 
    n_components=10, 
    metric='euclidean',
    random=False,
    signature=True
)

all_bands = False
use_precomputed_signature = True

if all_bands:
    dedupe_data = data
    dedupe_slice_full = phasor.Deduplicator(
        phasor.slice_vec_bands(
            dedupe_data, 
            start=0, 
            end=5),  # Not actually all bands -- we store a total of 10 bands, but stop at 5.
        **kwargs)
elif use_precomputed_signature:
    dedupe_data = signature_data
    dedupe_slices = [phasor.Deduplicator.from_reduced(
        phasor.slice_vec_bands(
            dedupe_data, 
            start=i, 
            end=i + 1, 
            ndims=5),   # This is the number of *complex* dimensions, which is a little confusing. Need to fix!
        **kwargs)       # It will always be half the number of dimensions in the flattened vector.
    for i in range(5)]  # We stop at 5, just as in the "all bands" case.
else:
    dedupe_data = data
    dedupe_slices = [phasor.Deduplicator(
        phasor.slice_vec_bands(
            dedupe_data, 
            start=i, 
            end=i + 1,
            ndims=300)  # This parameter already defaults to 300, so this is redundant. Making it explicit
        **kwargs)       # to reiterate that these are *complex* dimensions.
    for i in range(5)]  # We stop at 5, just as in the "all bands" case.

dedupe_boolean = phasor.Deduplicator(dedupe_slices[0])

for ds in dedupe_slices[1:5]:
    dedupe_boolean.merge(ds)

In [9]:
dedupe_data.shape

(20000, 100)

In [10]:
radius = 1.3

pairs_boolean = dedupe_boolean(radius)
pairs_single = dedupe_slices[0](radius)
# pairs_full = dedupe_slice_full(radius)
print("Number of candidates found by each test")
print()
print("Boolean test:     ", len(pairs_boolean))
print("Single-band test: ", len(pairs_single))
# print("Full-band test:   ", len(pairs_full))

Number of candidates found by each test

Boolean test:      3928
Single-band test:  10774


In [11]:
likely_true_positives_boolean = set(
    frozenset((a, b)) for a, b in pairs_boolean
    if metadata['title'][a] == metadata['title'][b]
)
likely_true_positives_single = set(
    frozenset((a, b)) for a, b in pairs_single
    if metadata['title'][a] == metadata['title'][b]
)
# likely_true_positives_full = set(
#     frozenset((a, b)) for a, b in pairs_full
#     if metadata['title'][a] == metadata['title'][b]
# )

print("Number of likely duplicates (based on title) found by each test")
print()
print("Boolean test:     ", len(likely_true_positives_boolean))
print("Single-band test: ", len(likely_true_positives_single))
# print("Full-band test:   ", len(likely_true_positives_full))
print()
print("Number of identically-titled volumes missed by single test, caught by boolean test:")
print(len(likely_true_positives_boolean - likely_true_positives_single))
print()
print("Number of identically-titled volumes missed by boolean test, caught by single test:")
print(len(likely_true_positives_single - likely_true_positives_boolean))
print()
print("NOTE: Many false positives will still appear in these counts because "
      "different volumes from multi-volume works may have the same title even "
      "though they do not contain the same content. This accounts for many of the "
      "matches captured by the single-band but not by the boolean test. The "
      "single-band test captures the broad semantic similarity between volumes of "
      "the same work, but can't make fine-grained distinctions between individual "
      "volumes of the work. A single ten-volume work can produce as many as one "
      "hundred false positives here, so this can give the impression that the "
      "boolean test has missed many duplicates. Hand check a few and you'll "
      "probably see that it hasn't.")

Number of likely duplicates (based on title) found by each test

Boolean test:      818
Single-band test:  1276

Number of identically-titled volumes missed by single test, caught by boolean test:
0

Number of identically-titled volumes missed by boolean test, caught by single test:
458

NOTE: Many false positives will still appear in these counts because different volumes from multi-volume works may have the same title even though they do not contain the same content. This accounts for many of the matches captured by the single-band but not by the boolean test. The single-band test captures the broad semantic similarity between volumes of the same work, but can't make fine-grained distinctions between individual volumes of the work. A single ten-volume work can produce as many as one hundred false positives here, so this can give the impression that the boolean test has missed many duplicates. Hand check a few and you'll probably see that it hasn't.


In [12]:
pairs = list(pairs_boolean)

def get_root(links, ix):
    while links[ix] != ix:
        ix = links[ix]
    return ix

def set_root(links, ix, root):
    while links[ix] != ix:
        old_ix = ix
        ix = links[ix]
        links[old_ix] = root
    links[ix] = root

def cluster_pairs(pairs, maxn=None):
    if maxn is None:
        maxn = max(x for p in pairs for x in p) + 1
        
    dupe_link = list(range(maxn))
    for a, b in pairs:
        a_root = get_root(dupe_link, a)
        set_root(dupe_link, a, a_root)
        set_root(dupe_link, b, a_root)
    
    for i in range(len(dupe_link)):
        dupe_link[i] = get_root(dupe_link, i)

    dupe_clusters = {c: [] for c in dupe_link}
    for i, c in enumerate(dupe_link):
        dupe_clusters[c].append(i)

    return list(dupe_clusters.values())

dupe_clusters = cluster_pairs(pairs, len(dedupe_data))
print('Total number of items:', sum(len(c) for c in dupe_clusters))
print('Number of clusters:', len(dupe_clusters))
print('Largest cluster:', max(len(c) for c in dupe_clusters))
print('Number of one-item clusters:', sum(len(c) == 1 for c in dupe_clusters))

Total number of items: 20000
Number of clusters: 17429
Largest cluster: 100
Number of one-item clusters: 16180


In [13]:
def sort_cluster(indices, data):
    centroid = sum(data[i] for i in indices) / len(indices)
    distances = [((data[i] - centroid) ** 2).sum() ** 0.5 for i in indices]
    order = sorted(range(len(distances)), key=distances.__getitem__)
    indices_sorted = [indices[o] for o in order]
    distances = [distances[o] for o in order]
    return list(zip(indices_sorted, distances))

def mean_dist(cluster_dist):
    return sum(d for c, d in cluster_dist) / len(cluster_dist)

# sort_data = phasor.PhasorSignature().fit_transform(data)
sort_data = dedupe_data

display_clusters = [sort_cluster(dc, sort_data)
                    for dc in dupe_clusters if len(dc) > 1]
display_clusters.sort(key=mean_dist)

for i, dc in enumerate(display_clusters):
    print()
    print(f'Cluster {i}, {len(dc)} items:')
    for j, (vol, dist) in enumerate(dc):
        vol_id = metadata.index[vol]
        vol_ti = metadata['title'][vol]
        vol_au = metadata['author'][vol]
        print('    Item', j, ' ~~  distance from cluster centroid:', dist)
        print('   ', vol_au, ' ~~ ', vol_ti)
        print('   ', phasor.htid_url(vol_id))
        print()



Cluster 0, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.04638300819908373
    Gaines, Ernest J., 1933-  ~~  Bloodline, by Ernest J. Gaines.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003682401

    Item 1  ~~  distance from cluster centroid: 0.04638300819908406
    Gaines, Ernest J., 1933-  ~~  Bloodline, by Ernest J. Gaines.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003682419


Cluster 1, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.04910712380150014
    Greene, Graham, 1904-1991.  ~~  The collected edition.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015002320029

    Item 1  ~~  distance from cluster centroid: 0.04910712380150027
    Greene, Graham, 1904-1991  ~~  Journey without maps / Graham Greene.
    https://babel.hathitrust.org/cgi/pt?id=inu.39000002700545


Cluster 2, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.052724566226568924
    Ellison, Harlan.  ~~  Dangerous visions : 33 original stories / edited by Ha

    Fletcher, Inglis, 1879-1969.  ~~  Bennett's welcome / by Inglis Fletcher.
    https://babel.hathitrust.org/cgi/pt?id=inu.30000111985713


Cluster 60, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.15722520711593962
    nan  ~~  Wonder tales / edited and introduced by Marina Warner ; with illustrations by Sophie Herxheimer.
    https://babel.hathitrust.org/cgi/pt?id=inu.30000044427619

    Item 1  ~~  distance from cluster centroid: 0.15722520711594018
    nan  ~~  Wonder tales : [six french stories of enchantment] / edited and introduced by Marina Warner ; with illustrations by Sophie Herxheimer ; [translated by Gilbert Adair ... et al.].
    https://babel.hathitrust.org/cgi/pt?id=inu.30000096418565


Cluster 61, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.15740379175808078
    Orwell, George, 1903-1950.  ~~  Coming up for air.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015000674310

    Item 1  ~~  distance from cluster centroid: 0.1574037917580810



Cluster 82, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.1882006984350708
    Balzac, Honoré de, 1799-1850.  ~~  La comédie humaine of Honoré de Balzac; tr. by Katharine Prescott Wormeley, illustrated by Jeanniot, [Laurent Desrousseaux, Adrien Moreau, etc.]
    https://babel.hathitrust.org/cgi/pt?id=ien.35556007878812

    Item 1  ~~  distance from cluster centroid: 0.18820069843507092
    Balzac, Honoré de, 1799-1850.  ~~  La comédie humaine of Honoré de Balzac / translated by Katharine Prescott Wormeley.
    https://babel.hathitrust.org/cgi/pt?id=inu.39000005960500


Cluster 83, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.18950459701864847
    Oliver, John Rathbone, 1872-1943.  ~~  Fear : the autobiography of James Edwards.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015002371238

    Item 1  ~~  distance from cluster centroid: 0.18950459701864858
    Oliver, John Rathbone, 1872-1943.  ~~  Fear : the autobiography of James Edwards.
    https://b

    Item 2  ~~  distance from cluster centroid: 0.3139687569979826
    Watson, Henry C. 1831-1869.  ~~  Camp-fires of the revolution: or, The War of independence,  illustrated by thrilling events and stories by the old continental soldiers, by Henry C. Watson. With original illustrations by Croome.
    https://babel.hathitrust.org/cgi/pt?id=loc.ark:/13960/t3223955f


Cluster 125, 3 items:
    Item 0  ~~  distance from cluster centroid: 0.25496282752323923
    Cather, Willa, 1873-1947.  ~~  My Ántonia, by Willa Cather, with illustrations by W. T. Benda.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015001146821

    Item 1  ~~  distance from cluster centroid: 0.26569630868407845
    Cather, Willa, 1873-1947.  ~~  My Ántonia / by Willa Sibert Cather ; with illustrations by W.T. Benda.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003659078

    Item 2  ~~  distance from cluster centroid: 0.26752542485488817
    Cather, Willa, 1873-1947.  ~~  My Ántonia, by Willa Cather, with ill

    Cary, Joyce, 1888-1957.  ~~  Prisoner of grace / Joyce Cary.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015000588239

    Item 1  ~~  distance from cluster centroid: 0.29285621806905926
    Cary, Joyce, 1888-1957.  ~~  Prisoner of grace / Joyce Cary.
    https://babel.hathitrust.org/cgi/pt?id=inu.39000004690165


Cluster 147, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.29458831281230174
    Prokosch, Frederic, 1908-1989.  ~~  The seven who fled.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015000606205

    Item 1  ~~  distance from cluster centroid: 0.2945883128123019
    Prokosch, Frederic, 1908-1989.  ~~  The seven who fled.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015000606213


Cluster 148, 2 items:
    Item 0  ~~  distance from cluster centroid: 0.298600354025258
    Spark, Muriel.  ~~  Robinson : a novel / by Muriel Spark.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015000609738

    Item 1  ~~  distance from cluster centroid: 0.29860035

...

    Johnson, Clifton, 1865-1940.  ~~  The country school, by Clifton Johnson; with illustrations by the author.
    https://babel.hathitrust.org/cgi/pt?id=loc.ark:/13960/t89g6528q


Cluster 1240, 31 items:
    Item 0  ~~  distance from cluster centroid: 1.783213180703772
    Richardson, Samuel, 1689-1761.  ~~  The history of Clarissa Harlowe,
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015002714825

    Item 1  ~~  distance from cluster centroid: 1.7979921333332922
    Richardson, Samuel, 1689-1761.  ~~  The novels of Samuel Richardson. Complete and unabridged.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015002715491

    Item 2  ~~  distance from cluster centroid: 1.8765354372453706
    Richardson, Samuel, 1689-1761.  ~~  Clarissa, a novel,
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003640821

    Item 3  ~~  distance from cluster centroid: 1.9228726598513748
    Richardson, Samuel, 1689-1761.  ~~  The novels of Samuel Richardson. Complete and unabridged.
    https:

    Item 17  ~~  distance from cluster centroid: 2.3653516284756546
    Fitzgerald, F. Scott 1896-1940.  ~~  Taps at reveille.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003343095

    Item 18  ~~  distance from cluster centroid: 2.392828756690393
    Munonye, John, 1929-  ~~  A kind of fool / John Munonye.
    https://babel.hathitrust.org/cgi/pt?id=inu.30000068570229

    Item 19  ~~  distance from cluster centroid: 2.4251757320561764
    Craig, Peter, 1969-  ~~  Blood father : [a novel] / Peter Craig.
    https://babel.hathitrust.org/cgi/pt?id=inu.30000095824532

    Item 20  ~~  distance from cluster centroid: 2.453392770975183
    Hutchinson, R. C. 1907-1975.  ~~  The unforgotten prisoner / R. C. Hutchinson.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015001763310

    Item 21  ~~  distance from cluster centroid: 2.488053557093455
    Gordimer, Nadine.  ~~  Occasion for loving.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003688259

    Item 22  ~~  distance fr

    nan  ~~  Great books of the Western World. Robert Maynard Hutchins, editor in chief.
    https://babel.hathitrust.org/cgi/pt?id=inu.39000005790287

    Item 16  ~~  distance from cluster centroid: 2.65911352097074
    Glasgow, Ellen Anderson Gholson, 1873-1945.  ~~  The miller of Old Church / by Ellen Glasgow
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003940908

    Item 17  ~~  distance from cluster centroid: 2.7710595710433887
    Noyes, Alfred, 1880-1958.  ~~  No other man : a novel / by Alfred Noyes.
    https://babel.hathitrust.org/cgi/pt?id=inu.32000000354227

    Item 18  ~~  distance from cluster centroid: 2.7778741490593815
    Tolstoy, Leo, graf, 1828-1910.  ~~  War and peace; the Maude translation, backgrounds and sources [and] essays in criticism. Edited by George Gibian.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015000099120

    Item 19  ~~  distance from cluster centroid: 3.026222504821093
    Tolstoy, Leo, graf, 1828-1910.  ~~  Anna Karenina; transla

    https://babel.hathitrust.org/cgi/pt?id=coo.31924014298123

    Item 34  ~~  distance from cluster centroid: 2.363213919156426
    Scott, Walter, Sir, 1771-1832.  ~~  Tales of the crusaders : with the author's last notes and additions.
    https://babel.hathitrust.org/cgi/pt?id=hvd.hwp6sh

    Item 35  ~~  distance from cluster centroid: 2.373128668603997
    Cooper, James Fenimore, 1789-1851.  ~~  Lionel Lincoln: or, The leaguer of Boston ... By the author of The spy" ..."
    https://babel.hathitrust.org/cgi/pt?id=hvd.hwjte4

    Item 36  ~~  distance from cluster centroid: 2.3769938565245625
    Scott, Walter, Sir, 1771-1832.  ~~  The monastery : a romance.
    https://babel.hathitrust.org/cgi/pt?id=hvd.32044086799616

    Item 37  ~~  distance from cluster centroid: 2.4196519181389244
    Scott, Walter, Sir, 1771-1832.  ~~  Waverley novels.
    https://babel.hathitrust.org/cgi/pt?id=hvd.hwaduv

    Item 38  ~~  distance from cluster centroid: 2.429128545366986
    Scott, Walter,

    Item 36  ~~  distance from cluster centroid: 2.602007926443135
    Fielding, Henry, 1707-1754.  ~~  The history of Amelia.
    https://babel.hathitrust.org/cgi/pt?id=hvd.hw3a9r

    Item 37  ~~  distance from cluster centroid: 2.6173781457888223
    Fielding, Henry, 1707-1754.  ~~  The works of Henry Fielding
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003342287

    Item 38  ~~  distance from cluster centroid: 2.6346215113097613
    Lytton, Rosina Bulwer Lytton, Baroness, 1802-1882.  ~~  Cheveley, or, The man of honour / by Lady Lytton Bulwer ; in two volumes.
    https://babel.hathitrust.org/cgi/pt?id=dul1.ark:/13960/t7pn9wz5m

    Item 39  ~~  distance from cluster centroid: 2.6364506437124917
    Fielding, Henry, 1707-1754.  ~~  The history of Tom Jones, a foundling. By Henry Fielding. With a memoir of the author, by Thomas Roscoe.
    https://babel.hathitrust.org/cgi/pt?id=mdp.39015003342733

    Item 40  ~~  distance from cluster centroid: 2.683026801548722
    Goldsm

In [14]:
data_freq_array = [phasor.unflatten_vec(r) / len(data) for r in data]  # 300 rows, 20 cols in each array, ~1000 arrays

data_freq_mean = data_freq_array[0]
for dfa in data_freq_array[1:]:
    data_freq_mean += dfa

NameError: name 'data' is not defined

In [ ]:
power_a = data_freq_mean[:, 1:].real
power_b = data_freq_mean[:, 1:].imag
power = (power_a * power_a + power_b * power_b) ** 0.5

mean_power = power.sum(axis=0) / 300
plt.plot(mean_power)

In [ ]:
scaled_mean = mean_power / mean_power.mean()
scaled_power = power / power.mean(axis=0)
# scaled_rel_diff = []

scaled_diffs = numpy.array([(((scaled_power[i] - scaled_mean) / scaled_power[i]) ** 2).sum()
                            for i in range(len(power))])

scaled_diffs_argsort = scaled_diffs.argsort()

for chunk in range(10):
    for i in range(chunk * 30, chunk * 30 + 30):
        plt.plot(power[scaled_diffs_argsort[i]] / 
                 power[scaled_diffs_argsort[i]].mean())
    plt.show()


